In [3]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed)
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(dataset[0])


HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)


In [1]:
import torch

# Parameters
batch_size = 2
num_negatives = 2
num_users = 3
num_items = 3
dim = 2

# Embeddings for users and items (3 users, 3 items, embedding dimension 2)
user_embedding = torch.tensor([[0.1, 0.2], [0.3, 0.4], [0.5, 0.6]])
item_embedding = torch.tensor([[0.7, 0.8], [0.9, 1.0], [1.1, 1.2]])

# h_index and t_index based on the edges and negative samples
# Each row represents a batch entry, and each column is a negative sample
# Here, `0` and `1` are valid head indices (users), and we corrupt them by keeping valid and invalid tails
h_index = torch.tensor([
    [0, 0, 0],  # For first edge (0, 0, 3) with two negatives
    [1, 2, 0]   # For second edge (1, 0, 4) with two negatives
])
t_index = torch.tensor([
    [3, 4, 5],  # Original (0, 0, 3), and corrupted tails [4, 2]
    [4, 4, 4]   # Original (1, 0, 4), and corrupted tails [5, 0]
])

# Gather head node embeddings
# (num_nodes, dim)
index_temp = h_index.unsqueeze(-1).expand(-1, -1, user_embedding.shape[-1])
h_embeddings = user_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, h_index.unsqueeze(-1).expand(-1, -1, dim))

# Adjust `t_index` to map to item IDs by subtracting `num_users`
index_temp = (t_index - num_users).clamp(min=0)
t_embeddings = item_embedding.unsqueeze(0).expand(batch_size,-1,-1).gather(1, index_temp.unsqueeze(-1).expand(-1, -1, dim))

print("User Embedding Tensor:\n", user_embedding)
print("Item Embedding Tensor:\n", item_embedding)
print("Head Embeddings:\n", h_embeddings)
print("Tail Embeddings:\n", t_embeddings)
print("size head Embeddings:\n", h_embeddings.shape)
print("size Tail Embeddings:\n", t_embeddings.shape)


User Embedding Tensor:
 tensor([[0.1000, 0.2000],
        [0.3000, 0.4000],
        [0.5000, 0.6000]])
Item Embedding Tensor:
 tensor([[0.7000, 0.8000],
        [0.9000, 1.0000],
        [1.1000, 1.2000]])
Head Embeddings:
 tensor([[[0.1000, 0.2000],
         [0.1000, 0.2000],
         [0.1000, 0.2000]],

        [[0.3000, 0.4000],
         [0.5000, 0.6000],
         [0.1000, 0.2000]]])
Tail Embeddings:
 tensor([[[0.7000, 0.8000],
         [0.9000, 1.0000],
         [1.1000, 1.2000]],

        [[0.9000, 1.0000],
         [0.9000, 1.0000],
         [0.9000, 1.0000]]])
size head Embeddings:
 torch.Size([2, 3, 2])
size Tail Embeddings:
 torch.Size([2, 3, 2])


In [ ]:
data = dataset[0]
plot_ratings_vs_time(data)
#print (data)
#print (data['user', 'rates', 'movie'].rating[:20])
#print (graph["movie"].x[:10,:])


In [7]:

movies_features = graph["movie"].x
print (movies_features[:4])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [8]:

user_features = graph["user"].x
print (movies_features[:10])

tensor([[0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])


In [8]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 

In [ ]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
%run script/run.py -c config/transductive/inference.yaml --dataset WN18RR --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

In [ ]:
%run script/run.py -c config/recommender/first_steps.yaml --dataset MovieLens100k --epochs 1 --bpe 10 --gpus "[0]" --ckpt null

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
14:04:02   Random seed: 1024
14:04:02   Config file: config/recommender/first_steps.yaml
14:04:02   {'checkpoint': None,
 'dataset': {'class': 'MovieLens100k', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'embedding_item': {'hidden_dims': [24, 24], 'input_dim': 18},
           'embedding_user': {'hidden_dims': [24, 24], 'input_dim': 24},
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [40, 40, 40, 40, 40],
                            'input_dim': 40,
                            'layer_norm': True,
                            'message_func': 'distmult',
                     

rel graph is built


14:04:05   ------------------------------
14:04:05   Number of parameters: 38265
14:04:05   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:04:05   Epoch 0 begin


Load rspmm extension. This may take a while...


14:04:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:04:06   binary cross entropy: 0.713987
14:04:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:04:06   Epoch 0 end
14:04:06   ------------------------------
14:04:06   average binary cross entropy: 0.663968
14:04:06   Save checkpoint to model_epoch_1.pth
14:04:06   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:04:06   Evaluate on valid
14:04:20   mr: 262.665
14:04:20   mrr: 0.0392176
14:04:20   hits@1: 0.01275
14:04:20   hits@3: 0.03025
14:04:20   hits@10: 0.0715
14:04:20   Load checkpoint from model_epoch_1.pth
14:04:20   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
14:04:20   Evaluate on valid


In [ ]:
#train
%run script/run.py -c config/transductive/inference.yaml --dataset MovieLens100k --epochs 10 --bpe 100 --gpus "[0]" --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

In [1]:
import numpy as np
import torch

print(np.__version__)  # Check NumPy version
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available


1.24.3
2.1.0
True


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [14]:
import matplotlib.pyplot as plt
import torch

def plot_ratings_vs_time(data):
    # Extract `rating` and `time` for edges in `(user, rates, movie)`
    ratings = data['user', 'rates', 'movie'].rating
    times = data['user', 'rates', 'movie'].time

    # Ensure `ratings` and `times` are on CPU for plotting if necessary
    if ratings.is_cuda:
        ratings = ratings.cpu()
    if times.is_cuda:
        times = times.cpu()

    # Plot the scatter plot for rating vs. time
    plt.figure(figsize=(10, 6))
    plt.scatter(ratings, times, alpha=0.5, s=10, c='blue', label='Edges (user, rates, movie)')
    plt.xlabel('Rating')
    plt.ylabel('Time')
    plt.title('Scatter plot of Rating vs Time for (user, rates, movie) edges')
    plt.legend()
    plt.show()





11:22:31   generated new fontManager


In [1]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",

    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset

    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: pitri (pitri-eth-z-rich). Use `wandb login --relogin` to force relogin


acc,▂▁▇▆▆▇█▇
loss,█▃▃▃▄▃▃▁
acc,0.83106
loss,0.0692
